In [2]:
import stim
import pygsti
from pygsti.extras.idletomography.idtcore import stochastic_jac_alt, build_class_jacobian
from pygsti.extras.idletomography.pauliobjs import NQPauliState
import numpy as np

Question I need to answer is: when is trace(RPQP) - trace(RQ), where R, Q and P are paulis, and R and Q can be signed. (This comes from working through the jacobian for the stochastic terms by hand.

In [3]:
sto_jac , pairs=  stochastic_jac_alt(1)

In [12]:
pairs

[('+X', '+X'),
 ('+X', '+Y'),
 ('+X', '+Z'),
 ('-X', '+X'),
 ('-X', '+Y'),
 ('-X', '+Z'),
 ('+Y', '+X'),
 ('+Y', '+Y'),
 ('+Y', '+Z'),
 ('-Y', '+X'),
 ('-Y', '+Y'),
 ('-Y', '+Z'),
 ('+Z', '+X'),
 ('+Z', '+Y'),
 ('+Z', '+Z'),
 ('-Z', '+X'),
 ('-Z', '+Y'),
 ('-Z', '+Z')]

In [4]:
sto_jac

array([[ 0., -2., -2.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  2.,  2.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [-2.,  0., -2.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 2.,  0.,  2.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [-2., -2.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 2.,  2.,  0.]])

In [5]:
same_pairs = [pair for pair in pairs if pair[0][-1] == pair[1][-1]]

In [7]:
same_pairs

[('+X', '+X'),
 ('-X', '+X'),
 ('+Y', '+Y'),
 ('-Y', '+Y'),
 ('+Z', '+Z'),
 ('-Z', '+Z')]

In [6]:
build_class_jacobian('S',1)

4


{(('+X', '+X'), 'Y'): -2.0,
 (('-X', '+X'), 'Y'): 2.0,
 (('+X', '+X'), 'Z'): -2.0,
 (('-X', '+X'), 'Z'): 2.0,
 (('+Y', '+Y'), 'X'): -2.0,
 (('-Y', '+Y'), 'X'): 2.0,
 (('+Y', '+Y'), 'Z'): -2.0,
 (('-Y', '+Y'), 'Z'): 2.0,
 (('+Z', '+Z'), 'X'): -2.0,
 (('-Z', '+Z'), 'X'): 2.0,
 (('+Z', '+Z'), 'Y'): -2.0,
 (('-Z', '+Z'), 'Y'): 2.0}

In [13]:
sto_jac_orig = build_class_jacobian('S', 1)
#refactor this to match output of new code
sto_jac_orig_refactored_dict = {}
for pair in same_pairs:
    sto_jac_orig_refactored_dict[pair] = np.zeros((1,3))
    for (key, val) in sto_jac_orig.items():
        if key[0] == pair:
            if key[1] =='X':
                sto_jac_orig_refactored_dict[pair] += np.array([val, 0, 0])
            if key[1] =='Y':
                sto_jac_orig_refactored_dict[key[0]] += np.array([0, val, 0])
            if key[1] =='Z':
                sto_jac_orig_refactored_dict[key[0]] += np.array([0, 0, val])

sto_jac_orig_refactored = np.concatenate([array for array in sto_jac_orig_refactored_dict.values()], axis = 0)
print(sto_jac_orig_refactored_dict)
print(sto_jac_orig_refactored)

4
{('+X', '+X'): array([[ 0., -2., -2.]]), ('-X', '+X'): array([[0., 2., 2.]]), ('+Y', '+Y'): array([[-2.,  0., -2.]]), ('-Y', '+Y'): array([[2., 0., 2.]]), ('+Z', '+Z'): array([[-2., -2.,  0.]]), ('-Z', '+Z'): array([[2., 2., 0.]])}
[[ 0. -2. -2.]
 [ 0.  2.  2.]
 [-2.  0. -2.]
 [ 2.  0.  2.]
 [-2. -2.  0.]
 [ 2.  2.  0.]]


In [14]:
j=0
for i in range(len(pairs)):
    #only print the ones where the prep and meas are the same.
    if pairs[i][0][-1] != pairs[i][1][-1]:
        continue
    print(f'{pairs[i]}  {sto_jac[i,:]}  {sto_jac_orig_refactored[j, :]}')
    j+=1

('+X', '+X')  [ 0. -2. -2.]  [ 0. -2. -2.]
('-X', '+X')  [0. 2. 2.]  [0. 2. 2.]
('+Y', '+Y')  [-2.  0. -2.]  [-2.  0. -2.]
('-Y', '+Y')  [2. 0. 2.]  [2. 0. 2.]
('+Z', '+Z')  [-2. -2.  0.]  [-2. -2.  0.]
('-Z', '+Z')  [2. 2. 0.]  [2. 2. 0.]
